# OGGM glacier climate simulator

App to initialize a simple glacier and look how it develops under changing climate settings (mass balance profile).

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from oggm import cfg
cfg.initialize()

# OGGM models
from oggm.core.massbalance import LinearMassBalance
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline

# There are several solvers in OGGM core. We use the default one for this experiment
from functools import partial
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define some variables for model and plotting

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# default tools for plot
default_tools = ['save', 'wheel_zoom', 'box_zoom', 'reset']

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

# glacier range
max_glacier_range = np.linspace(glacier_bottom_height,
                                glacier_top_height, nx)

## Define ranges for slider:

In [ ]:
# ELA height
ELA_height_min = 2000
ELA_height_max = 4500
ELA_height_default = 3000
ELA_height_range = np.arange(ELA_height_min,
                             ELA_height_max + 1,
                             100)

# mass balance gradient 
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_range = np.arange(mb_gradient_min,
                              mb_gradient_max + 1,
                              1)

# years of calculation
years_min = 10
years_max = 500
years_default = 150
years_range = np.arange(years_min, years_max + 1, 10)

## Define global model variables:

In [ ]:
bed_h = []
mb_model = []
model = []
run_surface_heights = []
widths = []

## Define global curves:

In [ ]:
bed_rock_curve = []
glacier_height_curve = []
mb_subfigure = []
mb_curve = []
width_curve = []

## Define Panels

### Panel for initializing glacier bed

In [ ]:
def initializing_glacier_bed_button_click(arg=None):
    main_figure.event(button_name='glacier_bed_button',
                      clicks=init_glacier_bed_button.clicks)

In [ ]:
bed_shape = pn.widgets.Select(name='bed shape',
                              options=['rectangular', 'trapezoidal', 'parabolic'])

bed_rock_profile = pn.widgets.Select(name='bedrock profile',
                                     options=['linear', 'getting flatter', 'getting steeper', 'cliff'])

bed_rock_width = pn.widgets.Select(name='width along glacier',
                                   options=['constant', 'getting narrower', 'getting wider'])

init_glacier_bed_button = pn.widgets.Button(name='initializing glacier bed',
                                            button_type='primary')
init_glacier_bed_button.param.watch(initializing_glacier_bed_button_click, 'clicks');

bed_rock_panel = pn.Column(pn.Row(bed_shape, bed_rock_profile, bed_rock_width),
                           init_glacier_bed_button)

### Panel for running the model

In [ ]:
def run_model_button_click(arg=None):
    main_figure.event(button_name='run_model',
                      clicks=run_model_button.clicks)

In [ ]:
ELA_height = pn.widgets.DiscreteSlider(name='equilibrium line height',
                                       options=list(ELA_height_range),
                                       value=ELA_height_default)

mb_gradient = pn.widgets.DiscreteSlider(name='mass balance gradient',
                                        options=list(mb_gradient_range),
                                        value=mb_gradient_default)

years_run_model = pn.widgets.DiscreteSlider(name='years to run the model',
                                            options=list(years_range),
                                            value=years_default)

run_model_button = pn.widgets.Button(name='run the model',
                                     button_type='primary')
run_model_button.param.watch(run_model_button_click, 'clicks');

run_model_panel = pn.Column(pn.Row(ELA_height, mb_gradient, years_run_model), run_model_button)

### add all panels together as tabs

In [ ]:
tab_menu = pn.Tabs(('change glacier bed', bed_rock_panel),
                   ('run glacier model', run_model_panel))

## Define function for creating curves

In [ ]:
def get_bed_rock_curve():
    global bed_h
    
    # look which bed rock profile was chosen
    if bed_rock_profile.value == 'linear':
        # change to fixed values top and bottom height
        bed_h = np.linspace(glacier_top_height, glacier_bottom_height, nx)
        
    elif bed_rock_profile.value == 'getting flatter':
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])
        
    elif bed_rock_profile.value == 'getting steeper':
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) +1.3) *
                          glacier_top_height for x in np.arange(nx)])
        
    elif bed_rock_profile.value == 'cliff':
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400
        
        bed_h = np.concatenate((np.linspace(glacier_top_height, cliff_top, nx/2),
                                np.linspace(cliff_bottom, glacier_bottom_height, nx/2)))
    
    # create curve of bed rock
    bed_rock_curve = hv.Area((distance_along_glacier, bed_h),
                             'distance along glacier (km)', 'altitude (m)',
                            ).opts(default_tools=default_tools,
                                                    color='lightgray',
                                                    line_alpha=0)
    
    return bed_rock_curve

In [ ]:
def get_glacier_height_curve():
    global model
    global run_surface_heights
    
    ELA = int(ELA_height.value)
    
    glacier_height_curve = (hv.Area((distance_along_glacier, run_surface_heights[4]),
                                    'distance along glacier (km)', 'altitude (m)',
                                    label='new').opts(default_tools=default_tools,
                                                              color='white',
                                                              line_alpha=0) *
                            hv.Curve((distance_along_glacier, run_surface_heights[0]),
                                     'distance along glacier (km)', 'altitude (m)',
                                     label='old').opts(default_tools=default_tools,
                                                               color='black') *
                            hv.Curve((distance_along_glacier, run_surface_heights[1]),
                                     'distance along glacier (km)', 'altitude (m)',
                                     label='one').opts(default_tools=default_tools,
                                                              line_dash='dashed',
                                                              color='red') *
                            hv.Curve((distance_along_glacier, run_surface_heights[2]),
                                     'distance along glacier (km)', 'altitude (m)',
                                     label='two').opts(default_tools=default_tools,
                                                            line_dash='dashed',
                                                            color='green') *
                            hv.Curve((distance_along_glacier, run_surface_heights[3]),
                                     'distance along glacier (km)', 'altitude (m)',
                                     label='three').opts(default_tools=default_tools,
                                                            line_dash='dashed',
                                                            color='orange') *
                            hv.Curve((distance_along_glacier, run_surface_heights[4]),
                                     'distance along glacier (km)', 'altitude (m)',
                                     label='new').opts(default_tools=default_tools,
                                                               color='blue') *
                            hv.Curve((distance_along_glacier, np.repeat(ELA, np.size(distance_along_glacier))),
                                     'distance along glacier (km)',
                                     'altitude (m)'
                                    ).opts(default_tools=default_tools,
                                           line_dash='dashed',
                                           line_width=3,
                                           color='black')
                           )
    
    return glacier_height_curve

In [ ]:
def get_mb_curve():
    global mb_model
    
    ELA = int(ELA_height.value)
    gradient = int(mb_gradient.value)
    
    mb_model = LinearMassBalance(ELA, grad=gradient)
    
    annual_mb = mb_model.get_annual_mb(max_glacier_range) * cfg.SEC_IN_YEAR
    
    mb_curve = (hv.Area((annual_mb, np.repeat(glacier_top_height, np.size(annual_mb))),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass gain'
                       ).opts(default_tools=default_tools,
                              color='lightgreen',
                              line_alpha=0) *
                hv.Area((annual_mb, np.repeat(ELA, np.size(annual_mb))),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass loss'
                       ).opts(default_tools=default_tools,
                              color='lightcoral',
                              line_alpha=0) *
                hv.Curve((annual_mb, max_glacier_range),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='mass balance'
                        ).opts(default_tools=default_tools,
                               color='black') * 
                hv.Curve((annual_mb, np.repeat(ELA, np.size(annual_mb))),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='ELA'
                        ).opts(default_tools=default_tools,
                               line_dash='dashed',
                               line_width=3,
                               color='black')
               )
    
    return mb_curve

In [ ]:
def get_width_curve():
    global widths
    global width_curve
    
    # look which glacier width was selected
    if bed_rock_width.value == 'constant':
        widths = np.zeros(nx) + 4.
        
    elif bed_rock_width.value == 'getting wider':
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])
        
    elif bed_rock_width.value == 'getting narrower':
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])
    
    # define lower and upper y limit which is shown
    y_min = -5 # in m
    y_max = 3
    
    width_curve = (hv.Area((distance_along_glacier, y_min),
                           'distance along glacier (km)',
                           'width (m)',
                           label='boarder of glacier rock bed'
                          ).opts(default_tools=default_tools,
                                 color='darkgray',
                                 line_alpha=0) * 
                   hv.Area((distance_along_glacier, y_max),
                           'distance along glacier (km)',
                           'width (m)',
                           label='boarder of glacier rock bed'
                          ).opts(default_tools=default_tools,
                                 color='darkgray',
                                 line_alpha=0) * 
                   hv.Area((distance_along_glacier, -widths/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='glacier rock bed'
                          ).opts(default_tools=default_tools,
                                 color='lightgray',
                                 line_alpha=0) * 
                   hv.Area((distance_along_glacier, widths/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='glacier rock bed'
                          ).opts(default_tools=default_tools,
                                 color='lightgray',
                                 line_alpha=0) * 
                   hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                            'distance along glacier (km)',
                            'width (m)',
                            label='center Flowline'
                           ).opts(default_tools=default_tools,
                                  color='black')
                   ).opts(ylim=(y_min, y_max))

    return width_curve

## Define functions for intialize flowline and run model

In [ ]:
def init_flowline_model():
    global model
    global bed_h
    
    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with choosen shape of glacier
    if bed_shape.value == 'rectangular':
        model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                                widths=widths, map_dx=map_dx)
        
    elif bed_shape.value == 'trapezoidal':
        model_flowline = TrapezoidalBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                                widths=widths, map_dx=map_dx)
        
    elif bed_shape.value == 'parabolic':
        model_flowline = ParabolicBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                              widths=widths, map_dx=map_dx)
    
    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0.)

In [ ]:
def run_the_model(total_years=0):
    global model
    global run_surface_heights
    
    # initialize model with old extend and new mass balance model
    model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.)
    
    # determine glacier state at starting point
    run_surface_heights = np.array(model.fls[-1].surface_h, ndmin=2)
    
    # if function is called with no given years or 0 years all steps of the 
    # glacier evolution are the same, otherwise calculate 5 evolution steps
    # with same amount of years in between each other
    if total_years==0:
        run_surface_heights = np.repeat(run_surface_heights, 5, axis=0)
    else:
        dyears = total_years / 4
        # run the model for each step and safe the surface height
        for years in np.arange(dyears, total_years + 1, dyears):
            model.run_until(years)
            run_surface_heights = np.append(run_surface_heights, 
                                            np.array(model.fls[-1].surface_h, ndmin=2), axis=0)

## Main function for dynamic map

In [ ]:
def change_plot(button_name='no', clicks=0):
    # use global curves
    global bed_rock_curve
    global glacier_height_curve
    global mb_curve
    global width_curve
    
    if button_name=='no':
        #TODO: set all widgets to default values
        bed_rock_profile.value = 'linear'
        bed_shape = 'rectangular'
        
        #initialisation of figure
        bed_rock_curve = get_bed_rock_curve()
        width_curve = get_width_curve()
        init_flowline_model()
        mb_curve = get_mb_curve()
        run_the_model()
        glacier_height_curve = get_glacier_height_curve()
        

    elif button_name=='glacier_bed_button':
        bed_rock_curve = get_bed_rock_curve()
        bed_rock_profile.value = 'linear'
        width_curve = get_width_curve()
        init_flowline_model()
        mb_curve = get_mb_curve()
        run_the_model()
        glacier_height_curve = get_glacier_height_curve()
        
    elif button_name=='run_model':
        mb_curve = get_mb_curve()
        run_the_model(total_years=years_run_model.value)
        glacier_height_curve = get_glacier_height_curve()
        
         
    return ((glacier_height_curve * bed_rock_curve).opts(legend_position='bottom_left',
                                                         legend_cols=2,
                                                         xaxis='top',
                                                         bgcolor='lightblue',
                                                         frame_width=580,
                                                         frame_height=200) + 
            (mb_curve).opts(legend_position='right',
                            xaxis='top',
                            yaxis='right',
                            frame_width=200,
                           frame_height=200) +
            (width_curve).opts(legend_cols=3,
                               legend_position='bottom_left',
                               frame_height=150,
                               frame_width=580,
                               yformatter='%8.0f')
           ).cols(2)

streamer and dynamic map

In [ ]:
main_stream = Stream.define('main_stream', button_name='no', clicks=0)

main_figure = hv.DynamicMap(change_plot, streams=[main_stream()])

new plot

In [ ]:
pn.Column(tab_menu, main_figure).servable()

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.